In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
def scrape_webpage(url):
    html=requests.get(url).text
    soup=BeautifulSoup(html, "html.parser")
    paragraphs=soup.find_all("p")
    text="\n".join([p.get_text() for p in paragraphs])
    return text

In [ ]:
import numpy as np
import faiss
import google.generativeai as genai

genai.configure(api_key="")

def build_faiss(chunks):
    vectors = []
    
    for c in chunks:
        v = genai.embed_content(
            model="models/text-embedding-004",
            content=c
        )["embedding"]
        vectors.append(v)
    
    matrix = np.array(vectors).astype("float32")
    dim = matrix.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(matrix)
    return index, matrix

def search(query, index, chunks, k=3):
    q_emb=